# Data voorbewerking

De data voor dit project is afkomstig van het back office systeem van Financial Markets, deels aangevuld met extra data vanuit Bloomberg.
De data bestaat uit de volgende elementen:

- Bond data
- Bondprijzen
- Government Yield curves
- Inflation data

Alle data is extracted en opgeslagen in csv files. In dit workbook lopen we door de data voorbereiding heen. Alle hier genoemde stappen kunnen ook geautomatiseerd worden uitgevoerd door het shell command 'Make Data'.

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import pandas as pd

sys.path.insert(0, "..") 
from src.data import make_dataset
from src.features import build_features

# Bond data

In [2]:

# Get bond data, drop unneeded columns, convert formats and strip training blanks
df_bonds = make_dataset.get_bond_data()

2021-12-31 15:41:44.205 | INFO     | src.data.make_dataset:get_bond_data:34 - Load bond data
2021-12-31 15:41:44.206 | INFO     | src.data.make_dataset:read_csv:19 - Loading data from ..\data\raw\bonds.csv


In [3]:
df_bonds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   ccy                225 non-null    string        
 1   country            225 non-null    string        
 2   issue_dt           225 non-null    datetime64[ns]
 3   first_coupon_date  202 non-null    datetime64[ns]
 4   mature_dt          225 non-null    datetime64[ns]
 5   isin               225 non-null    string        
 6   issuer_name        225 non-null    string        
 7   coupon_frq         225 non-null    string        
 8   coupon             225 non-null    float64       
 9   tot_issue          225 non-null    float64       
 10  cfi_code           218 non-null    string        
 11  issue_rating       220 non-null    string        
dtypes: datetime64[ns](3), float64(2), string(7)
memory usage: 21.2 KB


In [4]:
df_bonds.head()

,ccy,country,issue_dt,first_coupon_date,mature_dt,isin,issuer_name,coupon_frq,coupon,tot_issue,cfi_code,issue_rating
0,EUR,Netherlands,2009-02-13,2009-07-15,2019-07-15,NL0009086115,STAAT DER NEDERLANDEN,ANNUAL,4.000,5.000000e+09,DBFTFN,AAA
1,NLG,Austria,1994-02-28,1995-02-28,2024-02-28,NL0000133924,AUSTRIA,ANNUAL,6.250,1.000000e+09,DBFTXB,AA+
2,EUR,Netherlands,2012-03-09,2013-01-15,2033-01-15,NL0010071189,STAAT DER NEDERLANDEN,ANNUAL,2.500,4.160000e+09,DBFXXN,AAA
3,USD,United States,2009-05-15,2009-11-15,2019-05-15,US912828KQ20,UNITED STATES TREASURY,SEMI ANNUAL,3.125,6.441100e+10,<NA>,<NA>
4,USD,United States,2010-02-15,2010-08-15,2020-02-15,US912828MP29,UNITED STATES TREASURY,SEMI ANNUAL,3.625,0.000000e+00,<NA>,<NA>


Imputeren ontbrekende waarden voor issue rating. 
Issue rating wordt waar deze ontbreekt ingevuld met de meest voorkomende issue rating voor de issuer.

CFI code wordt waar deze ontbreekt ingevuld met code 'onbekend' = DXXXXX.

Datums met de waarde 1899-12-30 zijn default waardes van het bronsysteem. Deze worden verwijderd.
Daar waar de eerste coupon datum ontbreekt (o.a. zero coupon bonds) - wordt deze aangevuld met de issue datum.
De reden hiervoor is dat we zo zonder veel moeite de looptijd van de bond kunnen berekenen.

In [5]:
df_bonds = make_dataset.impute_bonds(df_bonds)

2021-12-31 15:41:44.581 | INFO     | src.data.make_dataset:impute_bonds:67 - Impute bond data


In [6]:
make_dataset.save_pkl('bonds', df_bonds)

2021-12-31 15:41:44.689 | INFO     | src.data.make_dataset:save_pkl:329 - Save preprocessed bonds data


In [7]:
df_bonds.count()

ccy                  225
country              225
issue_dt             225
first_coupon_date    225
mature_dt            225
isin                 225
issuer_name          225
coupon_frq           225
coupon               225
tot_issue            225
cfi_code             225
issue_rating         225
bond_duration        225
dtype: int64

# Bondprijzen

In [8]:
df_price = make_dataset.get_price()

2021-12-31 15:41:44.918 | INFO     | src.data.make_dataset:get_price:104 - Load bond price data
2021-12-31 15:41:44.920 | INFO     | src.data.make_dataset:read_csv:19 - Loading data from ..\data\raw\price.csv


In [9]:
df_price = make_dataset.impute_price(df_price)

2021-12-31 15:41:45.552 | INFO     | src.data.make_dataset:impute_price:129 - Impute bond price


In [10]:
df_price.head()

,reference_identifier,ccy,rate_dt,mid
0,BE0000332412,EUR,2014-01-20,100.719
1,BE0000332412,EUR,2014-01-24,101.005
2,BE0000332412,EUR,2014-01-28,100.953
3,BE0000332412,EUR,2014-01-22,100.359
4,BE0000332412,EUR,2014-01-21,100.601


In [11]:
df_price.describe()

,mid
count,209276.000000
mean,111.289025
std,15.522802
min,84.429000
25%,102.285000
50%,106.729000
75%,112.926000
max,195.749000


In [12]:
df_price.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 209276 entries, 0 to 220236
Data columns (total 4 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   reference_identifier  209276 non-null  string        
 1   ccy                   209276 non-null  string        
 2   rate_dt               209276 non-null  datetime64[ns]
 3   mid                   209276 non-null  float64       
dtypes: datetime64[ns](1), float64(1), string(2)
memory usage: 8.0 MB


In [13]:
make_dataset.save_pkl('price', df_price)

2021-12-31 15:41:46.140 | INFO     | src.data.make_dataset:save_pkl:329 - Save preprocessed price data


# Government Yield curves

In [14]:
df_yield = make_dataset.get_yield()

2021-12-31 15:41:49.134 | INFO     | src.data.make_dataset:get_yield:137 - Load goverment yield curve data
2021-12-31 15:41:49.135 | INFO     | src.data.make_dataset:read_csv:19 - Loading data from ..\data\raw\yield.csv


In [15]:
df_yield = make_dataset.impute_yield(df_yield)

2021-12-31 15:41:50.478 | INFO     | src.data.make_dataset:impute_yield:166 - Impute yield curve


In [16]:
df_yield.head()

,country,rate_dt,timeband,ratename,ccy,actual_dt,datedays,bid,offer,int_basis
1,France,2016-06-01,1 YEAR,GOV Yield Curve FR BB,EUR,2017-06-05,369,-0.5200,-0.498,ANNUAL
2,France,2016-06-01,10 YEARS,GOV Yield Curve FR BB,EUR,2026-06-03,3654,0.4755,0.475,ANNUAL
3,France,2016-06-01,15 YEARS,GOV Yield Curve FR BB,EUR,2031-06-03,5480,0.4755,0.475,ANNUAL
4,France,2016-06-01,2 YEARS,GOV Yield Curve FR BB,EUR,2018-06-04,733,-0.4597,-0.442,ANNUAL
5,France,2016-06-01,20 YEARS,GOV Yield Curve FR BB,EUR,2036-06-03,7307,1.2151,1.220,ANNUAL


In [17]:
df_yield.describe()

,datedays,bid,offer
count,89750.000000,89750.000000,89750.000000
mean,3389.135610,0.284506,0.273997
std,2877.657017,0.921237,0.921592
min,367.000000,-1.038000,-1.051000
25%,1463.000000,-0.429000,-0.443000
50%,2560.000000,0.016000,0.009000
75%,3657.000000,0.756000,0.750000
max,10968.000000,4.128000,4.109000


In [18]:
make_dataset.save_pkl('yield', df_yield)

2021-12-31 15:41:50.909 | INFO     | src.data.make_dataset:save_pkl:329 - Save preprocessed yield data


# Inflation data


In [19]:
df_inflation = make_dataset.get_inflation()  

2021-12-31 15:41:53.571 | INFO     | src.data.make_dataset:get_inflation:178 - Load goverment yield curve data
2021-12-31 15:41:53.572 | INFO     | src.data.make_dataset:read_csv:19 - Loading data from ..\data\raw\DE Inflation.csv
2021-12-31 15:41:53.589 | INFO     | src.data.make_dataset:read_csv:19 - Loading data from ..\data\raw\FR Inflation.csv
2021-12-31 15:41:53.605 | INFO     | src.data.make_dataset:read_csv:19 - Loading data from ..\data\raw\ES Inflation.csv
2021-12-31 15:41:53.622 | INFO     | src.data.make_dataset:read_csv:19 - Loading data from ..\data\raw\IT Inflation.csv
2021-12-31 15:41:53.639 | INFO     | src.data.make_dataset:read_csv:19 - Loading data from ..\data\raw\US Inflation.csv


In [20]:
df_inflation = make_dataset.impute_inflation(df_inflation)


df_inflation.info()

2021-12-31 15:42:08.258 | INFO     | src.data.make_dataset:impute_inflation:205 - Impute inflation curve


<class 'pandas.core.frame.DataFrame'>
Int64Index: 215007 entries, 0 to 216873
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   country    215007 non-null  string        
 1   rate_dt    215007 non-null  datetime64[ns]
 2   timeband   215007 non-null  string        
 3   inflation  215007 non-null  float64       
 4   ratename   215007 non-null  string        
dtypes: datetime64[ns](1), float64(1), string(3)
memory usage: 9.8 MB


In [21]:
make_dataset.save_pkl('inflation', df_inflation)

2021-12-31 15:42:08.426 | INFO     | src.data.make_dataset:save_pkl:329 - Save preprocessed inflation data


In [22]:
df_bpi = make_dataset.join_inflation(df_bonds, df_price, df_inflation)
df_bpy = make_dataset.join_yield(df_bonds, df_price, df_yield)
df_bpiy = make_dataset.fulljoin(df_bonds, df_price, df_inflation, df_yield)

C:\Users\M\AppData\Local\pypoetry\Cache\virtualenvs\timeseries-NkPdx04N-py3.8\lib\site-packages\pandas\core\frame.py:9190: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left,2 on the right)
  return merge(


In [23]:
df_bpiy = build_features.add_duration(df_bpiy)
df_bpiy = build_features.add_bid_offer_spread(df_bpiy)

In [24]:
df_bpy.to_csv(f'../data/processed/bpy.csv')
df_bpi.to_csv(f'../data/processed/bpi.csv')
df_bpiy.to_csv(f'../data/processed/bpiy.csv')

All in one make statement...

In [25]:
make_dataset.make_data()

2021-12-31 15:42:34.358 | INFO     | src.data.make_dataset:get_bond_data:34 - Load bond data
2021-12-31 15:42:34.359 | INFO     | src.data.make_dataset:read_csv:19 - Loading data from ..\data\raw\bonds.csv
2021-12-31 15:42:34.421 | INFO     | src.data.make_dataset:impute_bonds:67 - Impute bond data
2021-12-31 15:42:34.427 | INFO     | src.data.make_dataset:save_pkl:329 - Save preprocessed bonds data
2021-12-31 15:42:34.442 | INFO     | src.data.make_dataset:get_price:104 - Load bond price data
2021-12-31 15:42:34.443 | INFO     | src.data.make_dataset:read_csv:19 - Loading data from ..\data\raw\price.csv
2021-12-31 15:42:34.954 | INFO     | src.data.make_dataset:impute_price:129 - Impute bond price
2021-12-31 15:42:34.974 | INFO     | src.data.make_dataset:save_pkl:329 - Save preprocessed price data
2021-12-31 15:42:37.863 | INFO     | src.data.make_dataset:get_yield:137 - Load goverment yield curve data
2021-12-31 15:42:37.864 | INFO     | src.data.make_dataset:read_csv:19 - Loading d